In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"

# File url
file_url = '../datasets/everydayhealth.txt'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i 'scripts.py'

In [2]:
frame = pd.read_csv(file_url, sep='|', header= None, names = csv_header)
frame.head()

,tweetid,date,title
0,304596701757464576,Thu Feb 21 14:21:27 +0000 2013,#FastFood Makes Up 11 Percent of #Calories in ...
1,304595191329853441,Thu Feb 21 14:15:27 +0000 2013,"10 snacks to help you lose weight, burn fat, a..."
2,304587659018371072,Thu Feb 21 13:45:31 +0000 2013,10 foods that boost your skin AND slim your wa...
3,304580073380524032,Thu Feb 21 13:15:22 +0000 2013,What a heart attack feels like in women (it's ...
4,304572560270573569,Thu Feb 21 12:45:31 +0000 2013,#McDonalds oatmeal has almost 7 teaspoons of s...


In [3]:
frame.shape

(3239, 3)

In [4]:
frame.loc[:,'title'][4]

"#McDonalds oatmeal has almost 7 teaspoons of sugar in it! More healthy fast food that isn't: http://bit.ly/GGM5bA"

In [5]:
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(lambda row: split_txt_form_url(regex_url, row))
frame.head()

<re.Match object; span=(57, 77), match='http://bit.ly/VymaIc'>
http://bit.ly/VymaIc
<re.Match object; span=(63, 83), match='http://bit.ly/TeQPqk'>
http://bit.ly/TeQPqk
<re.Match object; span=(55, 76), match='http://trib.al/LDbxW6'>
http://trib.al/LDbxW6
<re.Match object; span=(69, 89), match='http://bit.ly/InrWrC'>
http://bit.ly/InrWrC
<re.Match object; span=(93, 113), match='http://bit.ly/GGM5bA'>
http://bit.ly/GGM5bA
<re.Match object; span=(55, 76), match='http://trib.al/4eRjME'>
http://trib.al/4eRjME
<re.Match object; span=(94, 114), match='http://bit.ly/yWptJ7'>
http://bit.ly/yWptJ7
<re.Match object; span=(55, 75), match='http://bit.ly/R7OpqA'>
http://bit.ly/R7OpqA
<re.Match object; span=(100, 120), match='http://bit.ly/HxUJEn'>
http://bit.ly/HxUJEn
<re.Match object; span=(85, 105), match='http://bit.ly/HXuSpU'>
http://bit.ly/HXuSpU
<re.Match object; span=(76, 96), match='http://bit.ly/yTBw80'>
http://bit.ly/yTBw80
<re.Match object; span=(34, 54), match='http://bit.ly/xbkHC0'>
http

<re.Match object; span=(37, 57), match='http://bit.ly/A1qD8J'>
http://bit.ly/A1qD8J
<re.Match object; span=(46, 66), match='http://bit.ly/VILMC0'>
http://bit.ly/VILMC0
<re.Match object; span=(59, 80), match='http://bit.ly/14NyAyM'>
http://bit.ly/14NyAyM
<re.Match object; span=(70, 91), match='http://trib.al/pTYApa'>
http://trib.al/pTYApa
<re.Match object; span=(80, 101), match='http://trib.al/GBRxuH'>
http://trib.al/GBRxuH
<re.Match object; span=(62, 83), match='http://bit.ly/14Ns8Yt'>
http://bit.ly/14Ns8Yt
<re.Match object; span=(109, 129), match='http://bit.ly/UwpIci'>
http://bit.ly/UwpIci
<re.Match object; span=(47, 67), match='http://bit.ly/HexAZv'>
http://bit.ly/HexAZv
<re.Match object; span=(48, 69), match='http://trib.al/3u1bQB'>
http://trib.al/3u1bQB
<re.Match object; span=(53, 74), match='http://bit.ly/10hUwTj'>
http://bit.ly/10hUwTj
<re.Match object; span=(70, 90), match='http://bit.ly/HCTYdo'>
http://bit.ly/HCTYdo
<re.Match object; span=(47, 67), match='http://bit.ly/Hb47Qb'

None
None
<re.Match object; span=(105, 125), match='http://bit.ly/WU23EF'>
http://bit.ly/WU23EF
<re.Match object; span=(53, 74), match='http://trib.al/2YFuD1'>
http://trib.al/2YFuD1
<re.Match object; span=(28, 49), match='http://trib.al/mLXN8K'>
http://trib.al/mLXN8K
<re.Match object; span=(78, 96), match='http://ow.ly/hlBIl'>
http://ow.ly/hlBIl
<re.Match object; span=(117, 135), match='http://ow.ly/hlBs3'>
http://ow.ly/hlBs3
<re.Match object; span=(86, 106), match='http://bit.ly/R7OMSc'>
http://bit.ly/R7OMSc
<re.Match object; span=(87, 108), match='http://bit.ly/13131l1'>
http://bit.ly/13131l1
<re.Match object; span=(116, 134), match='http://ow.ly/hlBlK'>
http://ow.ly/hlBlK
<re.Match object; span=(37, 57), match='http://bit.ly/Sx9MqS'>
http://bit.ly/Sx9MqS
<re.Match object; span=(30, 51), match='http://trib.al/OJmJLj'>
http://trib.al/OJmJLj
<re.Match object; span=(68, 88), match='http://bit.ly/H49ku3'>
http://bit.ly/H49ku3
<re.Match object; span=(75, 96), match='http://trib.al/xZweYA'

<re.Match object; span=(49, 69), match='http://bit.ly/IcCRF0'>
http://bit.ly/IcCRF0
<re.Match object; span=(66, 86), match='http://bit.ly/HIM5pn'>
http://bit.ly/HIM5pn
<re.Match object; span=(55, 75), match='http://bit.ly/H71XTI'>
http://bit.ly/H71XTI
<re.Match object; span=(46, 66), match='http://bit.ly/wLismZ'>
http://bit.ly/wLismZ
<re.Match object; span=(58, 78), match='http://bit.ly/GHpWdj'>
http://bit.ly/GHpWdj
<re.Match object; span=(65, 86), match='http://trib.al/WYCQxz'>
http://trib.al/WYCQxz
<re.Match object; span=(57, 77), match='http://bit.ly/yyC84F'>
http://bit.ly/yyC84F
<re.Match object; span=(83, 103), match='http://bit.ly/SHHR7E'>
http://bit.ly/SHHR7E
<re.Match object; span=(45, 66), match='http://trib.al/9Pldmh'>
http://trib.al/9Pldmh
<re.Match object; span=(67, 87), match='http://bit.ly/Sx9PD0'>
http://bit.ly/Sx9PD0
<re.Match object; span=(36, 57), match='http://trib.al/JnzWMb'>
http://trib.al/JnzWMb
<re.Match object; span=(37, 78), match='http://paper.li/EverydayHealt

http://paper.li/EverydayHealth/1340975022
<re.Match object; span=(34, 54), match='http://bit.ly/SJyvUc'>
http://bit.ly/SJyvUc
<re.Match object; span=(26, 46), match='http://bit.ly/H4Kbj8'>
http://bit.ly/H4Kbj8
<re.Match object; span=(115, 136), match='http://trib.al/kBfvmF'>
http://trib.al/kBfvmF
<re.Match object; span=(44, 65), match='http://trib.al/Vo1Ca7'>
http://trib.al/Vo1Ca7
<re.Match object; span=(45, 65), match='http://bit.ly/yosdlo'>
http://bit.ly/yosdlo
<re.Match object; span=(74, 94), match='http://bit.ly/y0UEie'>
http://bit.ly/y0UEie
<re.Match object; span=(26, 47), match='http://trib.al/rKq13M'>
http://trib.al/rKq13M
<re.Match object; span=(52, 73), match='http://trib.al/oEDvgy'>
http://trib.al/oEDvgy
<re.Match object; span=(50, 70), match='http://bit.ly/VhiIjs'>
http://bit.ly/VhiIjs
<re.Match object; span=(100, 121), match='http://trib.al/Izfjva'>
http://trib.al/Izfjva
<re.Match object; span=(46, 67), match='http://trib.al/rUynXv'>
http://trib.al/rUynXv
<re.Match object; 

...

http://trib.al/3KP1Ln
<re.Match object; span=(124, 142), match='http://ow.ly/hyJzH'>
http://ow.ly/hyJzH
<re.Match object; span=(101, 119), match='http://ow.ly/hyIXE'>
http://ow.ly/hyIXE
<re.Match object; span=(80, 100), match='http://bit.ly/V4zlk0'>
http://bit.ly/V4zlk0
<re.Match object; span=(49, 69), match='http://bit.ly/VkgOCH'>
http://bit.ly/VkgOCH
<re.Match object; span=(35, 56), match='http://bit.ly/10bcBSV'>
http://bit.ly/10bcBSV
<re.Match object; span=(39, 60), match='http://trib.al/qaTIu7'>
http://trib.al/qaTIu7
<re.Match object; span=(63, 84), match='http://trib.al/rfzt9u'>
http://trib.al/rfzt9u
<re.Match object; span=(61, 81), match='http://bit.ly/ULxCPJ'>
http://bit.ly/ULxCPJ
<re.Match object; span=(52, 73), match='http://trib.al/k2qCO0'>
http://trib.al/k2qCO0
<re.Match object; span=(72, 92), match='http://bit.ly/XJcFsn'>
http://bit.ly/XJcFsn
<re.Match object; span=(80, 100), match='http://bit.ly/VQCMuJ'>
http://bit.ly/VQCMuJ
<re.Match object; span=(52, 72), match='http://b

http://bit.ly/SFricA
<re.Match object; span=(37, 78), match='http://paper.li/EverydayHealth/1340975022'>
http://paper.li/EverydayHealth/1340975022
<re.Match object; span=(35, 55), match='http://bit.ly/Kfn98u'>
http://bit.ly/Kfn98u
<re.Match object; span=(67, 87), match='http://bit.ly/z1kw5w'>
http://bit.ly/z1kw5w
<re.Match object; span=(93, 113), match='http://bit.ly/ITTq42'>
http://bit.ly/ITTq42
<re.Match object; span=(48, 70), match='http://SeniorHomes.com'>
http://SeniorHomes.com
<re.Match object; span=(50, 70), match='http://bit.ly/xswY8x'>
http://bit.ly/xswY8x
None
<re.Match object; span=(58, 78), match='http://bit.ly/GEqi2F'>
http://bit.ly/GEqi2F
<re.Match object; span=(83, 103), match='http://bit.ly/QKs2Hz'>
http://bit.ly/QKs2Hz
<re.Match object; span=(50, 71), match='http://trib.al/wODt3R'>
http://trib.al/wODt3R
<re.Match object; span=(47, 68), match='http://trib.al/zfQW29'>
http://trib.al/zfQW29
<re.Match object; span=(124, 209), match='http://www.everydayhealth.com/healthtalk

<re.Match object; span=(61, 82), match='http://bit.ly/14adE5I'>
http://bit.ly/14adE5I
<re.Match object; span=(50, 70), match='http://bit.ly/QKbpvI'>
http://bit.ly/QKbpvI
None
<re.Match object; span=(107, 128), match='http://bit.ly/14e04NS'>
http://bit.ly/14e04NS
None
None
None
None
None
None
None
None
None
None
None
None
<re.Match object; span=(66, 86), match='http://bit.ly/FPZ1XZ'>
http://bit.ly/FPZ1XZ
None
None
None
<re.Match object; span=(116, 136), match='http://bit.ly/YqkzY3'>
http://bit.ly/YqkzY3
None
None
None
None
None
<re.Match object; span=(69, 87), match='http://ow.ly/h6s6U'>
http://ow.ly/h6s6U
None
None
None
<re.Match object; span=(111, 131), match='http://bit.ly/SE23bp'>
http://bit.ly/SE23bp
None
<re.Match object; span=(24, 48), match='http://TalkPsoriasis.org'>
http://TalkPsoriasis.org
None
None
None
None
<re.Match object; span=(118, 147), match='http://www.psoriasis.org/iwfm'>
http://www.psoriasis.org/iwfm
None
None
None
<re.Match object; span=(52, 72), match='http://bit

None
None
None
None
None
None
None
None
None
None
None
None
<re.Match object; span=(93, 113), match='http://bit.ly/XdEceY'>
http://bit.ly/XdEceY
None
None
None
None
None
None
None
None
None
<re.Match object; span=(106, 126), match='http://bit.ly/A3zxlB'>
http://bit.ly/A3zxlB
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
<re.Match object; span=(45, 65), match='http://bit.ly/yoJh7c'>
http://bit.ly/yoJh7c
None
<re.Match object; span=(35, 55), match='http://bit.ly/ytLAdX'>
http://bit.ly/ytLAdX
None
None
<re.Match object; span=(67, 87), match='http://bit.ly/GGFvSs'>
http://bit.ly/GGFvSs
<re.Match object; span=(52, 73), match='http://bit.ly/10h3lNc'>
http://bit.ly/10h3lNc
<re.Match object; span=(40, 60), match='http://bit.ly/U3WJxE'>
http://bit.ly/U3WJxE
<re.Match object; span=(47, 67), match='http://bit.ly/U3WItv'>
http://bit.ly/U3WItv
<re.Match object; span=(115, 133), match='http://ow.ly/gTL1B'>
http://ow.ly/gTL1B
<re.Ma

,tweetid,date,title,url
0,304596701757464576,Thu Feb 21 14:21:27 +0000 2013,#FastFood Makes Up 11 Percent of #Calories in ...,http://bit.ly/VymaIc
1,304595191329853441,Thu Feb 21 14:15:27 +0000 2013,"10 snacks to help you lose weight, burn fat, a...",http://bit.ly/TeQPqk
2,304587659018371072,Thu Feb 21 13:45:31 +0000 2013,10 foods that boost your skin AND slim your wa...,http://trib.al/LDbxW6
3,304580073380524032,Thu Feb 21 13:15:22 +0000 2013,at a heart attack feels like in women (it's di...,http://bit.ly/InrWrC
4,304572560270573569,Thu Feb 21 12:45:31 +0000 2013,#McDonalds oatmeal has almost 7 teaspoons of s...,http://bit.ly/GGM5bA


In [10]:
frame.loc[:,'url'][0]

'http://bit.ly/VymaIc'

In [ ]:
#Author:  //a[@class="article_link"]/span/text()
#Content: //div[@class="article-body"]/p/text()

In [41]:
url = 'http://bit.ly/VymaIc'

# Start the connection
res = requests.get(url)
print(res.status_code)

# Initialize BeautifulSoup()
soup = BeautifulSoup(res.content, 'lxml')

# Initialize XPath
# xpath_selector = Selector(text=res.text)

# Get return content as text
# text_html = res.text
author = soup.find('a',{'class':'article_link'}).text
print("Author: ", author)
content = soup.find('div',{'class':'article-body'})
print("Content: ", content.text)

200
Author:    Steven Reinberg, HealthDay News 
Content:  THURSDAY, Feb. 21, 2013 (HealthDay News) — Fast food fare from restaurants such as McDonald's, Burger King, Wendy's and Pizza Hut accounted for more than 11 percent of the calories in American adults' daily diets in recent years, federal health officials reported Thursday.Yet that's lower than it was from 2003 to 2006, when fast food contributed an average of nearly 13 percent of daily calories to the American diet, said report author Cheryl Fryar, of the U.S. Centers for Disease Control and Prevention.Whether this is an ongoing trend isn't clear, Fryar added. Several factors were examined, to see how they related to how many daily calories U.S. adults got from fast food between 2007 and 2010."We could see a difference by age group," Fryar noted. "As you get older, the percentage of calories from fast food declines."One expert sees this as a positive trend."The good news from this study is that as we get older, perhaps we do get

In [43]:
author = author
# ide = author.index(",")
# print(ide)
print(author[:author.index(",")])

  Steven Reinberg


In [37]:
s="mystring"
s.index("r")

4

In [6]:
# # Split Url out of content
# def get_url_test(regex, sample):
#     result = re.search(regex, sample)[0]

#     if result:
#         print(result)
#         return result
#     else:
#         print("[ERROR]: ",re.search(regex, sample))


#     return ''

In [8]:
# test_string ='''
#     304298842495528961|Wed Feb 20 18:37:52 +0000 2013|RT @foundersheart: @RMichlerMD @MontefioreNYC @EverydayHealth symptoms that aren't exactly the "Hollywood heart attack" but r just as deadly #HealthTalk #GoRed
# '''

# get_url_test(regex_url, test_string)